In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import final_project.config as c
import final_project.logic.Utils as u
from final_project.logic.Utils import Axis, EulerAngle as EA
from final_project.models.Trajectory import Trajectory

# "#1b7837", "#de77ae", "#762a83"
COLORS = ["gold", "cyan", "lightgreen", 'k']

In [2]:
# load data:
rel_pnp_cameras = pd.read_pickle(os.path.join(c.DATA_WRITE_PATH, "pnp_cameras.pkl"))
abs_pnp_cameras = pd.Series(u.convert_to_absolute_cameras(rel_pnp_cameras), index=rel_pnp_cameras.index, name=rel_pnp_cameras.name)
pnp_traj = Trajectory.from_absolute_cameras(abs_pnp_cameras)

rel_ba_cameras = pd.read_pickle(os.path.join(c.DATA_WRITE_PATH, "ba_cameras.pkl"))
abs_ba_cameras = pd.Series(u.convert_to_absolute_cameras(rel_ba_cameras), index=rel_ba_cameras.index, name=rel_ba_cameras.name)
ba_traj = Trajectory.from_absolute_cameras(abs_ba_cameras)

rel_pg_cameras = pd.read_pickle(os.path.join(c.DATA_WRITE_PATH, "pg_cameras.pkl"))
abs_pg_cameras = pd.Series(u.convert_to_absolute_cameras(rel_pg_cameras), index=rel_pg_cameras.index, name=rel_pg_cameras.name)
pg_traj = Trajectory.from_absolute_cameras(abs_pg_cameras)

rel_gt_cameras = pd.Series(u.read_ground_truth_cameras(use_relative=True), index=rel_pnp_cameras.index, name="gt_cameras")
abs_gt_cameras = pd.Series(u.read_ground_truth_cameras(use_relative=False), index=rel_gt_cameras.index, name=rel_gt_cameras.name)
gt_traj = Trajectory.from_absolute_cameras(abs_gt_cameras)

In [6]:
def calculate_movement_along_axis(abs_cams: pd.Series, axis: Axis) -> pd.Series:
    t = Trajectory.from_absolute_cameras(abs_cams)
    coords = t.coordinates  # shape (3, N) repr. locations relative to first Camera/Frame
    real_movement = np.diff(coords, 1)  # shape (3, N-1) repr. actual movement on each axis, relative to previous location
    if axis == Axis.ALL:
        axis_movement = np.linalg.norm(real_movement, ord=2, axis=0)  # shape (N-1,) repr. distance travelled (L2) between successive Frames
    else:
        axis_movement = real_movement[axis.value]  # shape (N-1,) repr. distance travelled on a single axis (X/Y/Z) between successive Frames
    s = pd.Series(axis_movement, index=abs_cams.index[:-1], name=axis.name)
    return s

def calculate_distance_along_axis(est_cams, gt_cams, axis: Axis) -> pd.Series:
    assert len(est_cams) == len(gt_cams)
    est = Trajectory.from_absolute_cameras(est_cams)
    gt = Trajectory.from_absolute_cameras(gt_cams)
    coords_diff = est.coordinates - gt.coordinates
    if axis != Axis.ALL:
        distances = coords_diff[axis.value]
    else:
        distances = np.linalg.norm(coords_diff, ord=2, axis=0)
    res = pd.Series(distances, name=axis.name, index=est_cams.index)
    return res

def running_sum(s: pd.Series, window: int) -> pd.Series:
    num_vals = len(s)
    assert window > 0, "window size must be positive"
    assert num_vals > window, "window size cannot be bigger than input series"
    res = s.rolling(window).sum().dropna()
    res.index = res.index - res.index.min()  # index should start at 0
    assert res.shape == (num_vals - window + 1,), f"{num_vals} - {window} +1 != {res.shape}"
    return res

In [80]:
gt_x_movement = calculate_movement_along_axis(abs_gt_cameras, Axis.X)
gt_x_cum_movement = running_sum(abs(gt_x_movement), 100)

pnp_x_err_rel = calculate_distance_along_axis(rel_pnp_cameras, rel_gt_cameras, Axis.X)
pnp_x_cum_err_rel = running_sum(abs(pnp_x_err_rel), 100)

df = pd.concat([gt_x_cum_movement, pnp_x_cum_err_rel, 100 * pnp_x_cum_err_rel / gt_x_cum_movement], axis=1)
s = df.iloc[:, -1]
print(f"{s.mean():.2f}\t{s.std():.2f}\t{s.median():.2f}\t{s.min():.2f}\t{s.max():.2f}")
df

11.33	23.55	3.08	0.53	175.20


,X,X,X
FrameIdx,,,
0,24.596661,1.018393,4.140369
1,25.387212,1.036153,4.081398
2,26.177500,1.035116,3.954221
3,26.973901,1.038119,3.848606
4,27.778316,1.044704,3.760862
...,...,...,...
3346,59.635690,1.202214,2.015930
3347,60.493929,1.159933,1.917437
3348,61.348803,1.150495,1.875334


In [79]:
gt_x_movement = calculate_movement_along_axis(abs_gt_cameras, Axis.X)
gt_x_cum_movement = running_sum(abs(gt_x_movement), 800)

pnp_x_movement = calculate_movement_along_axis(abs_pnp_cameras, Axis.X)
pnp_x_cum_movement = running_sum(abs(pnp_x_movement), 800)

pnp_x_err = calculate_distance_along_axis(abs_pnp_cameras, abs_gt_cameras, Axis.X)
pnp_x_cum_err = running_sum(pnp_x_err, 800)

pnp_x_err_rel = calculate_distance_along_axis(rel_pnp_cameras, rel_gt_cameras, Axis.X)
pnp_x_cum_err_rel = running_sum(abs(pnp_x_err_rel), 800)

df = pd.concat([gt_x_cum_movement, pnp_x_cum_movement, pnp_x_cum_err, pnp_x_cum_err_rel, 100 * pnp_x_cum_err_rel / gt_x_cum_movement], axis=1)
s = df.iloc[:, -1]
print(f"{s.mean():.2f}\t{s.std():.2f}\t{s.median():.2f}\t{s.min():.2f}\t{s.max():.2f}")
df

2.81	0.49	2.79	1.69	4.50


,X,X,X,X,X
FrameIdx,,,,,
0,252.183951,248.692582,-654.244956,8.464369,3.356427
1,252.808865,249.328753,-654.547538,8.478469,3.353707
2,253.442462,249.975414,-654.861376,8.476422,3.344515
3,254.091404,250.636748,-655.188278,8.467330,3.332395
4,254.753124,251.312660,-655.527573,8.465825,3.323149
...,...,...,...,...,...
2646,427.160801,418.544523,-18346.099628,11.671512,2.732346
2647,427.880876,419.172898,-18330.183740,11.613412,2.714170
2648,428.590967,419.759495,-18314.288103,11.524104,2.688835


In [60]:

dists = []
prev_cam = abs_gt_cameras[0]
for i in range(1, len(abs_gt_cameras)):
    curr_cam = abs_gt_cameras[i]
    cx, cy, cz = curr_cam.calculate_coordinates()
    px, py, pz = prev_cam.calculate_coordinates()
    abs_diff = abs(cx - px)
    dists.append(abs_diff)
    prev_cam = curr_cam
    print(abs_diff)
    if i==4:
        break

sum(dists[:800])

0.014581630839319648
0.019898302323314104
0.019012318085786815
0.014862535978745844


0.06835478722716641

In [58]:
dists = []
errs = []
prev_cam_gt = rel_gt_cameras[0]
for i in range(1, len(rel_gt_cameras)):
    curr_cam_gt = rel_gt_cameras[i]
    cx, cy, cz = curr_cam_gt.calculate_coordinates()
    dists.append(abs(cx))
    print(abs(cx))
    if i == 4:
        break

sum(dists[:800])

0.014581630839319555
0.019030441557441602
0.017818756201358962
0.012930434604152732


0.06436126320227285

In [62]:
t = Trajectory.from_absolute_cameras(rel_gt_cameras)
print(t.coordinates[0][1:5])

[-0.01458163 -0.01903044 -0.01781876 -0.01293043]


In [64]:
t2 = Trajectory.from_absolute_cameras(abs_gt_cameras)
diff = np.diff(t2.coordinates, 1)
print(diff[0][:5])

[-0.01458163 -0.0198983  -0.01901232 -0.01486254 -0.01280119]
